# CONFIG

In [ ]:
import main
import pandas as pd
import json

class Args:
    df_path = "benchmark\task3_variant_prediction\VariPred\example\dataset"
    pred = "target"
    output = "VariPred_output"
    train = False

args = Args()

# INFERENCE VARIPRED

In [ ]:
target_df = pd.read_csv(f"{args.df_path}/{args.pred}.csv")
main.run_VariPred(target_ds=args.pred, output=args.output)

# EVALUATION METRICS

In [ ]:
from calculate_metrics import evaluate_performance
res_file = 'benchmark\task3_variant_prediction\VariPred\example\output_results\VariPred_output.txt' 
ori_file = 'benchmark\task3_variant_prediction\VariPred\example\dataset\test.parquet'
    
metrics = evaluate_performance(res_file, ori_file)
print(json.dumps(metrics, indent=4))

with open('evaluation_metrics.json', 'w') as f:
    json.dump(metrics,indent=4)